In [114]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
import numpy as np

team_data = pd.read_csv("../Stats_competition-/final_opponent_and_team_data.csv")

In [115]:
threshold = 6

In [116]:
team_data

,Date,Team,Opponent,Location,WAB,ADJO,ADJD,EFF,EFG%,TO%,...,Opp EFG%,Opp TO%,Opp OR%,Opp FTR,Opp 2P,Opp 3P,Opponent_score,Team_score,opp_adj_o,opp_adj_d
0,2024-11-04,Duke,Maine,H,0.1,125.3,95.2,130.6,64.3,17.7,...,39.5,19.0,19.4,31.6,18-43,3-14,62,96,108.9,110.8
1,2024-11-08,Duke,Army,H,0.1,124.7,92.3,141.0,61.3,11.3,...,39.1,18.3,23.3,14.1,13-35,8-29,58,100,107.2,119.0
2,2024-11-12,Duke,Kentucky,N,-0.2,106.8,86.4,95.7,42.3,9.3,...,47.6,14.6,23.3,38.1,15-38,10-25,77,72,119.6,88.1
3,2024-11-16,Duke,Wofford,H,-0.1,124.7,56.9,133.7,61.3,15.5,...,28.9,29.5,29.3,5.3,9-24,5-33,35,86,69.8,115.5
4,2024-11-22,Duke,Arizona,A,0.6,111.7,75.9,101.9,50.0,20.7,...,45.3,22.2,16.7,20.8,15-30,6-23,55,69,98.9,90.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297,2024-11-27,Michigan,Xavier,N,0.3,110.9,73.9,109.9,58.2,14.1,...,41.4,26.8,25.6,15.5,12-35,8-23,53,78,85.8,96.0
298,2024-12-03,Michigan,Wisconsin,A,1.0,102.7,84.4,99.3,44.1,17.8,...,39.1,14.8,34.8,29.7,16-37,6-27,64,67,103.9,89.6
299,2024-12-07,Michigan,Iowa,H,1.4,121.1,99.5,114.5,58.6,22.9,...,47.4,5.4,30.6,22.1,23-50,9-27,83,85,120.5,104.8
300,2024-12-10,Michigan,Arkansas,N,0.9,123.6,108.4,112.3,64.4,21.9,...,56.8,18.1,33.3,33.3,24-43,9-23,89,87,124.8,102.1


In [117]:
team_data["Location"] = np.where(
    team_data["Location"] == "N", 0, np.where(team_data["Location"] == "H", 1, -1)
)

In [118]:
columns_to_convert = [
    "Location",
    "ADJO",
    "ADJD",
    "EFG%",
    "TO%",
    "OR%",
    "FTR",
    "Opp EFG%",
    "Opp TO%",
    "Opp OR%",
    "Opp FTR",
]
for col in columns_to_convert:
    team_data[col] = pd.to_numeric(team_data[col], errors="coerce")

In [119]:
team_data_cleaned = team_data.dropna()

X = team_data_cleaned[columns_to_convert]
y_team = team_data_cleaned["Team_score"]
y_opp = team_data_cleaned["Opponent_score"]

X_train, X_test, y_team_train, y_team_test = train_test_split(
    X, y_team, test_size=0.2, random_state=42
)
y_opp_train, y_opp_test = train_test_split(y_opp, test_size=0.2, random_state=42)

model_team = LinearRegression()
model_team.fit(X_train, y_team_train)
team_pred = model_team.predict(X_test)

model_opp = LinearRegression()
model_opp.fit(X_train, y_opp_train)
opp_pred = model_opp.predict(X_test)

team_rmse = mean_squared_error(y_team_test, team_pred, squared=False)
opp_rmse = mean_squared_error(y_opp_test, opp_pred, squared=False)
print(f"Team Score RMSE: {team_rmse}, Opponent Score RMSE: {opp_rmse}")

Team Score RMSE: 5.694257875441654, Opponent Score RMSE: 5.397831922401339


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [120]:
team_accuracy = (abs(team_pred - y_team_test) <= threshold).mean() * 100
opp_accuracy = (abs(opp_pred - y_opp_test) <= threshold).mean() * 100
print(f"Team Score Accuracy: {team_accuracy:.2f}%")
print(f"Opponent Score Accuracy: {opp_accuracy:.2f}%")

Team Score Accuracy: 68.85%
Opponent Score Accuracy: 73.77%


In [121]:
predict_games = pd.read_csv("../Stats_competition-/basketball_games_data.csv")

In [122]:
predict_games["Location"] = np.where(
    predict_games["Location"] == "Neutral",
    0,
    np.where(predict_games["Location"] == "Home", 1, -1),
)

In [123]:
predict_games

,date,Location,Team,Opponent,ADJO,ADJD,EFG%,TO%,OR%,FTR,Opp EFG%,Opp TO%,Opp OR%,Opp FTR,opp_adj_o,opp_adj_d
0,2024-12-22,1,Oklahoma,Central Arkansas,112.6,103.1,55.9,12.3,28.6,20.6,44.5,16.5,22.0,17.2,97.5,107.7
1,2024-12-22,-1,Illinois St.,Cornell,118.1,109.0,63.0,15.1,23.3,33.3,58.5,13.6,11.1,32.1,112.8,116.8
2,2024-12-22,0,Illinois,Missouri,110.9,93.4,50.9,22.9,36.4,40.4,42.2,12.1,23.1,63.8,118.0,98.2
3,2024-12-22,1,Colgate,Iona,104.5,125.5,63.3,29.2,34.6,32.7,54.8,20.4,42.9,25.4,110.5,111.1
4,2024-12-22,1,Alabama,Kent St.,111.2,77.8,52.5,25.0,45.5,58.3,29.3,14.5,28.6,17.3,80.8,89.7
5,2024-12-22,1,Iowa St.,Morgan St.,117.3,107.4,66.9,14.9,30.8,44.1,49.2,17.6,27.0,23.8,112.7,114.4
6,2024-12-22,-1,New Hampshire,Stonehill,115.7,134.2,57.9,14.6,29.7,47.4,59.0,13.1,38.7,36.1,121.4,141.4
7,2024-12-22,1,Michigan,Purdue Fort Wayne,115.8,80.5,71.2,25.7,45.0,10.2,38.5,8.6,13.3,13.8,95.2,112.1


In [124]:
X = predict_games[columns_to_convert]

In [125]:
X

,Location,ADJO,ADJD,EFG%,TO%,OR%,FTR,Opp EFG%,Opp TO%,Opp OR%,Opp FTR
0,1,112.6,103.1,55.9,12.3,28.6,20.6,44.5,16.5,22.0,17.2
1,-1,118.1,109.0,63.0,15.1,23.3,33.3,58.5,13.6,11.1,32.1
2,0,110.9,93.4,50.9,22.9,36.4,40.4,42.2,12.1,23.1,63.8
3,1,104.5,125.5,63.3,29.2,34.6,32.7,54.8,20.4,42.9,25.4
4,1,111.2,77.8,52.5,25.0,45.5,58.3,29.3,14.5,28.6,17.3
5,1,117.3,107.4,66.9,14.9,30.8,44.1,49.2,17.6,27.0,23.8
6,-1,115.7,134.2,57.9,14.6,29.7,47.4,59.0,13.1,38.7,36.1
7,1,115.8,80.5,71.2,25.7,45.0,10.2,38.5,8.6,13.3,13.8


In [126]:
from pygam import LinearGAM, s, f
from sklearn.metrics import mean_squared_error

# Fit PyGAM for Team Score
gam_team = LinearGAM(
    s(0) + s(1) + s(2) + s(3) + s(4) + s(5) + s(6) + s(7) + s(8) + s(9) + s(10)
).fit(X_train, y_team_train)
team_gam_pred = gam_team.predict(X_test)

# Fit PyGAM for Opponent Score
gam_opp = LinearGAM(
    s(0) + s(1) + s(2) + s(3) + s(4) + s(5) + s(6) + s(7) + s(8) + s(9) + s(10)
).fit(X_train, y_opp_train)
opp_gam_pred = gam_opp.predict(X_test)

# Calculate RMSE for PyGAM
team_gam_rmse = mean_squared_error(y_team_test, team_gam_pred, squared=False)
opp_gam_rmse = mean_squared_error(y_opp_test, opp_gam_pred, squared=False)
print(f"Team Score RMSE (PyGAM): {team_gam_rmse}")
print(f"Opponent Score RMSE (PyGAM): {opp_gam_rmse}")

# Calculate accuracy for PyGAM
team_gam_accuracy = (abs(team_gam_pred - y_team_test) <= threshold).mean() * 100
opp_gam_accuracy = (abs(opp_gam_pred - y_opp_test) <= threshold).mean() * 100
print(f"Team Score Accuracy (PyGAM): {team_gam_accuracy:.2f}%")
print(f"Opponent Score Accuracy (PyGAM): {opp_gam_accuracy:.2f}%")

# Predict scores using PyGAM
team_gam_pred_new = gam_team.predict(X)
opp_gam_pred_new = gam_opp.predict(X)

# Add PyGAM predictions to the table
predictions_gam = pd.DataFrame(
    {
        "Predicted Team Score GAM": team_gam_pred_new,
        "Predicted Opponent Score GAM": opp_gam_pred_new,
    }
)

predict_games = pd.concat([predict_games, predictions_gam], axis=1)

Team Score RMSE (PyGAM): 6.986779995501826
Opponent Score RMSE (PyGAM): 6.7824075879007
Team Score Accuracy (PyGAM): 65.57%
Opponent Score Accuracy (PyGAM): 68.85%


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [127]:
team_pred_new = model_team.predict(X)
opp_pred_new = model_opp.predict(X)

predictions = pd.DataFrame(
    {
        "Predicted Team Score LR": team_pred_new,
        "Predicted Opponent Score LR": opp_pred_new,
    }
)

In [128]:
predict_games = pd.concat([predict_games, predictions], axis=1)

In [129]:
predict_games[
    [
        "Team",
        "Opponent",
        "Predicted Team Score LR",
        "Predicted Opponent Score LR",
    ]
]

,Team,Opponent,Predicted Team Score LR,Predicted Opponent Score LR
0,Oklahoma,Central Arkansas,79.166199,61.675325
1,Illinois St.,Cornell,88.372032,80.128860
2,Illinois,Missouri,75.987238,72.419649
3,Colgate,Iona,71.799647,76.346761
4,Alabama,Kent St.,76.886418,49.444535
5,Iowa St.,Morgan St.,93.422177,70.027455
6,New Hampshire,Stonehill,83.245746,90.426563
7,Michigan,Purdue Fort Wayne,89.296138,56.119612


In [130]:
# Add columns for averages
predict_games["Average Predicted Team Score"] = predict_games[
    ["Predicted Team Score LR", "Predicted Team Score GAM"]
].mean(axis=1)

predict_games["Average Predicted Opponent Score"] = predict_games[
    ["Predicted Opponent Score LR", "Predicted Opponent Score GAM"]
].mean(axis=1)

# Select columns to display
columns_to_display = [
    "Team",
    "Opponent",
    "Predicted Team Score LR",
    "Predicted Opponent Score LR",
    "Predicted Team Score GAM",
    "Predicted Opponent Score GAM",
    "Average Predicted Team Score",
    "Average Predicted Opponent Score",
]

# Display the updated table
# print(predict_games[columns_to_display].to_string(index=False))
display(predict_games[columns_to_display])

,Team,Opponent,Predicted Team Score LR,Predicted Opponent Score LR,Predicted Team Score GAM,Predicted Opponent Score GAM,Average Predicted Team Score,Average Predicted Opponent Score
0,Oklahoma,Central Arkansas,79.166199,61.675325,84.631341,68.560955,81.898770,65.118140
1,Illinois St.,Cornell,88.372032,80.128860,92.261725,82.962965,90.316878,81.545912
2,Illinois,Missouri,75.987238,72.419649,75.333006,70.113170,75.660122,71.266409
3,Colgate,Iona,71.799647,76.346761,71.895929,81.946786,71.847788,79.146773
4,Alabama,Kent St.,76.886418,49.444535,75.871347,52.862724,76.378882,51.153630
5,Iowa St.,Morgan St.,93.422177,70.027455,93.749737,72.419655,93.585957,71.223555
6,New Hampshire,Stonehill,83.245746,90.426563,81.912131,88.573926,82.578938,89.500244
7,Michigan,Purdue Fort Wayne,89.296138,56.119612,85.642075,50.549227,87.469107,53.334420
